In [13]:
import csv
import json
import os
import requests
import pandas as pd
from dotenv import load_dotenv; load_dotenv()

True

Loading json file containing hotel data

In [14]:
json_file = "Bangkok.json"  # to edit
filepath = "/Users/pritish/projects/Hotels-Decision-Support-and-Sentimental-Analysis"

with open(os.path.join(filepath,json_file)) as file:
    data = json.load(file)

Requesting geocode data through geocodeapi using hotel addresses.

In [15]:
API_KEY = os.getenv('GEOCODE_API_KEY')
url = 'https://app.geocodeapi.io/api/v1/search'

for i in data.keys():
    address = data[i]['address']
    response = requests.get(url, params = {'apikey': API_KEY,'text': address}).json()
    # accessing coordinates
    features = response['features']
    geometry = features[0]['geometry']
    coord = geometry['coordinates']
    data[i]['geocode'] = coord
    print(i)

Element Kuala Lumpur Hotel
The RuMa Hotel and Residences, Kuala Lumpur
Four Seasons Hotel Kuala Lumpur
Mandarin Oriental, Kuala Lumpur Hotel
Banyan Tree Kuala Lumpur Hotel
Hilton Kuala Lumpur Hotel
Lanson Place Bukit Ceylon Serviced Residences Hotel, Kuala Lumpur
Grand Hyatt Kuala Lumpur Hotel
The Majestic Hotel Kuala Lumpur
Oasia Suites Kuala Lumpur by Far East Hospitality Hotel
Sunway Velocity Hotel, Kuala Lumpur
PARKROYAL Serviced Suites Kuala Lumpur Hotel
Sunway Putra Hotel, Kuala Lumpur
Traders Hotel, Kuala Lumpur
Royale Chulan Kuala Lumpur Hotel
Komune Living Hotel, Kuala Lumpur
Metro Hotel, Kuala Lumpur
Pullman Kuala Lumpur City Centre Hotel & Residences
Corus Hotel Kuala Lumpur
Seri Pacific Hotel Kuala Lumpur


Adding co-ordinates to JSON data files

In [16]:
with open(os.path.join(filepath,json_file), 'w') as outfile:
            json.dump(data, outfile, indent=4)

Extracting hotel names and geocodes into a Data Frame

In [28]:
to_add = []
columns = ['Hotel_Name','Longitude','Latitude']
for i in data.keys():
    values = [i, data[i]['geocode'][0], data[i]['geocode'][1]]
    zipped_dict = dict(zip(columns, values))
    to_add.append(zipped_dict)
# Making pandas dataframe
df = pd.DataFrame(to_add)
df.head()

,Hotel_Name,Longitude,Latitude
0,Element Kuala Lumpur Hotel,101.718394,3.158934
1,"The RuMa Hotel and Residences, Kuala Lumpur",101.721146,3.150499
2,Four Seasons Hotel Kuala Lumpur,113.833569,2.521947
3,"Mandarin Oriental, Kuala Lumpur Hotel",101.709852,3.149475
4,Banyan Tree Kuala Lumpur Hotel,101.713741,3.150451


Write geocode pertinent data to a csv file


In [29]:
output_filename = json_file[:-5]
with open(os.path.join(filepath,f'{output_filename}_hotelsGeocode.csv'), 'w', newline='') as file:
    df.to_csv(file)
